<a href="https://colab.research.google.com/github/Tbharvesh/DeepLearningProject/blob/main/DL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# !unzip -q dataset.zip
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
# %cd https://drive.google.com/drive/folders/11jXIcJ6MF3R_FbeMQigZ5PeQSEpYmurO?usp=drive_link


In [ ]:
#loading image datasets
train_dataset = image_dataset_from_directory(
    '/content/drive/MyDrive/Covid19-dataset/train',
    image_size=(256, 256),
    batch_size=32,
    )
test_dataset=image_dataset_from_directory(
    '/content/drive/My Drive/Covid19-dataset/test',
    image_size=(256, 256),
    batch_size=32)

Found 251 files belonging to 3 classes.
Found 66 files belonging to 3 classes.


In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),
])

# Applying data augmentation to dataset
augmented_train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))


In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
preprocess_input = Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
train_dataset
